# 02 Baseline

In [ ]:
import config
from utils import data_model
from utils.model import Model

import pandas as pd
import numpy as np
import tensorflow as tf

## Load data

In [ ]:
data, imgs_left, imgs_right = data_model.load(
    config.PATH_DATA_FEATURES01_DLIB_AUGMENTED_NORM_CSV,
    config.PATH_DATA_FEATURES01_DLIB_AUGMENTED_NORM_IMGS_LEFT,
    config.PATH_DATA_FEATURES01_DLIB_AUGMENTED_NORM_IMGS_RIGHT
)

In [ ]:
data.head()

## Split data

In [ ]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = data_model.split(
    data, imgs_left, imgs_right,
    train_size=0.95,
    validation_size=0.95,
    random_state=42
)

In [ ]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

## Model

### Architecture

In [ ]:
def get_model(features, left_imgs, right_imgs, *args, **kwargs):
    with tf.variable_scope('model'):
        left_flat = tf.contrib.layers.flatten (left_imgs)
        right_flat =  tf.contrib.layers.flatten (right_imgs)
        flat_input = tf.concat(
            values=[features],
            axis=1,
            name='concat'
        )
        # 12+(20x30)x2 = 1212 inputs
        d1 = tf.layers.dense(flat_input, 1024, activation=tf.nn.relu)
        d2 = tf.layers.dense(d1, 512, activation=tf.nn.relu)
        d3 = tf.layers.dense(d2, 128, activation=tf.nn.relu)
        d4 = tf.layers.dense(d3, 2, activation=None)
    return d4

### Parameters

In [ ]:
MODEL_NAME = 'baseline-05'

EPOCHS = 200
BATCH_SIZE = 1024
LEARNING_RATE = 0.0001

## Runs

In [ ]:
model = Model(MODEL_NAME, get_model)

### Train

In [ ]:
model.train(
    train_data, train_imgs_left, train_imgs_right,
    validation_data, validation_imgs_left, validation_imgs_right,
    BATCH_SIZE, EPOCHS, LEARNING_RATE
)

### Test

In [ ]:
model_test = Model(MODEL_NAME, saved_model=MODEL_NAME+".final")

In [ ]:
model_test.test(
    test_data, test_imgs_left, test_imgs_right
)

### Results


| Name | Epochs | Batch Size | Learning rate  | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| baseline-01 | 20 | 512 | 0.005 | 0.14687612652778625 | 0.1866748034954071 | 0.19118537 |
| baseline-02 | 25 | 256 | 0.001 | 0.07768817245960236 | 0.1376853734254837 | 0.13483852 |
| baseline-03 | 40 | 256 | 0.0001 | 0.048277597874403 | 0.14694760739803314 | 0.14895809  |
| baseline-04 | 100 | 512 | 0.0001 | 0.031881727278232574 | 0.14400143921375275 | 0.14583308 |
| baseline-05 | 200 | 1024 | 0.0001 | 0.028280340135097504 | 0.14690683782100677 | 0.15199703 |

#### First iteration

* Half of the augmented images had the wrong 'x' label.

| Name | Epochs | Batch Size | Learning rate  | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| baseline-01 | 20 | 512 | 0.005 | 0.1395065039396286 | 0.19760280847549438 | 0.19909994 |
| baseline-02 | 25 | 256 | 0.001 | 0.07893882691860199 | 0.15243780612945557 | 0.15074611 |
| baseline-02 | 25 | 256 | 0.001 | 0.07598952203989029 | 0.14176610112190247 | 0.14149468 |
| baseline-03 | 40 | 256 | 0.0001 | 0.057024769484996796 | 0.1646086573600769 | 0.16265412 |
| baseline-04 | 100 | 512 | 0.0001 | 0.03750447556376457 | 0.1642383337020874 | 0.16351545 |
| baseline-05 | 200 | 1024 | 0.0001 | 0.029262783005833626 | 0.1729333996772766 | 0.17122823 |